In [1]:
#This first line will show plots produced by matplotlib inside this Jupyter notebook
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
import netCDF4 as nc
import xarray as xr
import numpy as np
import pandas as pd
from dask.distributed import Client, progress
import cmocean as cm                              # Nice colormaps
import cartopy.crs as ccrs                        # For making maps with different projections
import cartopy.feature as cft        

In [1]:
import ZonalStatsFunctions as zsf
import cosima_cookbook as cc

varInt = 'mld'
session = cc.database.create_session()

SO_SST = zsf.getACCESSdata(varInt, '2008-01', '2009-12', freq = '1 monthly', ses = session)
SO_SSTc = zsf.corrlong(SO_SST)
SO_SSTc[0].plot()

usage: ZonalStatsFunctions.py [-h]

This script contains functions used to perform timeseries within different
sectors of the Southern Ocean.

optional arguments:
  -h, --help  show this help message and exit


In [1]:
client = Client(n_workers = 12)

In [3]:
session = cc.database.create_session()

In [4]:
#Set up variables
#Experiment
exp = "01deg_jra55v140_iaf_cycle2"

#Variable of interest
varInt = 'surface_temp' #SST

#Define frequency. Remember to check frequency and variable of interest are related to each other, for example, aice_m has a monthly frequency, while aice has a daily frequency.
freq = '1 monthly'

In [5]:
weights =  cc.querying.getvar(exp, 'area_t', session, frequency = 'static', n = -1)
weights.name = 'weights'
weights = weights.fillna(0)

In [6]:
vararray = cc.querying.getvar(exp, varInt, session, frequency = freq, start_time = '1965-01', end_time = '2019-01')
vararray = vararray.sel(time = slice('1965', '2018'))
vararray = vararray-273.15
vararray = vararray.groupby('time.year').mean('time')

In [7]:
weighted_var = vararray.weighted(weights)

In [8]:
weighted_mean = weighted_var.mean(("xt_ocean", "yt_ocean"))
# weighted_mean.compute()
weighted_mean

<xarray.DataArray 'surface_temp' (year: 54)>
dask.array<truediv, shape=(54,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * year     (year) int64 1965 1966 1967 1968 1969 ... 2014 2015 2016 2017 2018

In [ ]:
weighted_mean.plot()

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


In [8]:
# weights.plot()
# vararray
# vararray[0].plot()
# weighted_mean.plot()

,Array,Chunk
Bytes,1.96 GiB,1.48 MiB
Shape,"(54, 2700, 3600)","(1, 540, 720)"
Count,3132017 Tasks,1350 Chunks
Type,float32,numpy.ndarray
